# Peer-graded Assignment: Эксперименты с моделью

На прошлой неделе вы поучаствовали в соревновании на kaggle и, наверняка, большинство успешно справилось с прохождением baseline, а значит пора двигаться дальше - заняться оптимизацией модели, провести серию экспериментов и построить сильное финальное решения.

В этом задании вам нужно провести ряд эскпериментов, оценить качество полученных в процессе экспериментирования моделей и выбрать лучшее решение. 

Задание будет оцениваться на основании загруженного jupyther notebook и развернутых ответов на поставленные вопросы.

## Загрузка библиотек

In [1]:
import pandas as pd
import random
import numpy as np
from scipy.sparse import coo_matrix, hstack
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_curve, precision_recall_curve, f1_score, roc_auc_score, recall_score, precision_score, log_loss

from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

seed = 1903
first_categorial_index = 190

Объявим функции, необходимые для построения базовой модели

In [2]:
def fill_numericna(train_frame, test_frame, averageCalculator):
    """ Функция заполняет значения в числовом фрейме значениями, посчитанными averageCalculator. """
    
    # Посчитаем средние по колонкам
    numeric_avgs = averageCalculator(train_frame)
    
    # Оставим только те колонки, в которых среднее значение не равно NaN, т.к. в таких колонках совсем нет значений
    numeric_avgs = numeric_avgs.dropna()
    dropped_columns = train_frame.columns.drop(numeric_avgs.index)
    n_frame_train = train_frame[list(numeric_avgs.index)]
    n_frame_test = test_frame[list(numeric_avgs.index)]
    
    # Заполним пропущенные численные значения средними
    n_frame_train = n_frame_train.fillna(numeric_avgs, axis=0)
    n_frame_test = n_frame_test.fillna(numeric_avgs, axis=0)
    return (n_frame_train, n_frame_test, dropped_columns)

def fill_numericna_means(train_frame, test_frame):
    """ Функция заполняет значения в числовом фрейме средними и удаляет те колонки, в которых значений нет. """
    return fill_numericna(
        train_frame,
        test_frame,
        lambda f: f.mean(axis=0, skipna=True))

def remove_constant_features(frame, min_count=2):
    """Функция удаляет колонки, которые содержат только одно значение."""
    
    # Посчитаем количества уникальных значений по колонкам
    unique_counts = frame.nunique()
    # Удалим колонки с количеством значений меньшим min_count
    columns_to_drop = unique_counts[unique_counts < min_count].index
    
    return (frame.drop(columns=columns_to_drop), columns_to_drop)

class MatrixLabelEncoder:
    """ Класс кодирует категории числами от 0 до n, где n количество категорий в колонке. """
    
    def __init__(self):
        self.encoders = []
    
    def fit(self, matrix):
        for column_number in range(matrix.shape[1]):
            column = matrix[:,column_number]
            labelEncoder = LabelEncoder().fit(column)
            self.encoders.append(labelEncoder)
        return self
    
    def transform(self, matrix):
        transformed = np.empty(matrix.shape)
        for column_number in range(matrix.shape[1]):
            labelEncoder = self.encoders[column_number]
            num_column = labelEncoder.transform(matrix[:,column_number])
            for row_number, val in enumerate(num_column):
                transformed[row_number, column_number] = val
        return transformed
    
class CompositeEncoder:
    """ Класс принимает набор фабрик с методами fit и transform и последовательно применяет их """
    def __init__(self, encoder_factories):
        self.encoder_factories = encoder_factories
        
    def fit(self, matrix):
        encoders = []
        transformed = matrix
        for encoder_factory in self.encoder_factories:
            encoder = encoder_factory().fit(transformed)
            encoders.append(encoder)
            transformed = encoder.transform(transformed)
        self.encoders = encoders
        return self

    def transform(self, matrix):
        for encoder in self.encoders:
            matrix = encoder.transform(matrix)
        return matrix
    
def predict_ridge_proba(X, model):
    """ Функция возвращает вероятности предсказаний для класса churn модель Ridge """
    # Поскольку RidgeClassifier не обладает функцией predict_proba приходится считать его вручную
    # Подробнее можно посмотреть здесь:
    # https://www.codesd.com/item/scikit-learn-ridge-classifier-extract-class-probabilities.html
    func = model.decision_function(X)
    return np.exp(func) / (1 + np.exp(func))

def predict_model_proba(X, model):
    """ Функция возвращает вероятности предсказаний для класса churn """
    return list(zip(*model.predict_proba(X)))[1]
    
def stratifiedKFold_fscore(
    frame,
    labels,
    model_factory,
    process_frame,
    frame_to_matrix,
    numeric_features,
    categorial_features,
    predict_probabilities,
    seed,
    folds_count = 3):
    """ Функция разбивает набор данных на folds_count, считает ROC-AUC на каждом фолде
        и возвращает усредненное по фолдам значение.
        Функция также возвращает модель, показавшую лучшее качество, её метрики и разделение данных.
        Разделение данных нужно для того, чтобы строить метрики модели на данных, на которых она не обучалась."""
    skf = StratifiedKFold(n_splits=folds_count, shuffle=True, random_state=seed)
    
    best_model = None
    best_score = 0
    best_precision = 0
    best_recall = 0
    best_table = None
    best_split = None
    best_encoders = None
    best_dropped_columns = None
    metrics_sum = 0
    for train_indices, test_indices in skf.split(frame, labels):
        # Разобьем фрем на train и test с помощью функции process_frame
        # Внутри такой функции мы можем по-разному обрабатывать признаки обучаясь только на train наборе.
        train_frame, train_labels, test_frame, test_labels, dropped_numeric, dropped_categorial = process_frame(
            frame.loc[train_indices, :],
            labels.loc[train_indices, :],
            frame.loc[test_indices, :],
            labels.loc[test_indices, :],
            numeric_features,
            categorial_features)
        numeric_cleaned = numeric_features.drop(dropped_numeric)
        categorial_cleaned = categorial_features.drop(dropped_categorial)
        # Преобразуем фреймы в матрицы.
        # Тут можно выполнить финальное преобразование признаков, например масштабирование признаков.
        # В функции frame_to_matrix энкодеры типа StandardScaler обучаются только на train признаках.
        X_train, X_test, num_encoder, cat_encoder = frame_to_matrix(
            train_frame,
            test_frame,
            numeric_cleaned,
            categorial_cleaned)
        y_train = train_labels.as_matrix().flatten()
        y_test = test_labels.as_matrix().flatten()

        model = model_factory()
        # Обучим модель
        model.fit(X_train, y_train)
        
        # Построим вероятности принадлежности к целевому классу
        probabilities = predict_probabilities(X_test, model)
        # Считаем roc auc score
        rocAuc = roc_auc_score(y_test, probabilities)
        metrics_sum += rocAuc
        if(best_model is None or best_score < rocAuc):
            # В случае, если модель лучше предыдущих сохраним её
            # оценку, модель, матрицу ошибок и разделение данных
            best_score = rocAuc
            best_model = model
            best_dropped_columns = (dropped_numeric, dropped_categorial)
            best_encoders = (num_encoder, cat_encoder)
            best_split = (X_train, y_train, X_test, y_test)
    return (
        metrics_sum/folds_count,
        best_model,
        best_score,
        best_split,
        best_encoders,
        best_dropped_columns)

def cleanup_frame_common(frame, numeric_features, categorial_features):
    """Функция делит признакина числовые и категориальные и удаляет константные признаки, содержащие только одно значение"""
    # Разделим коллекции на группы - числовые и категориальные.
    numeric_frame = frame[numeric_features].copy()
    categorial_frame = frame[categorial_features].copy()
    # Удалим вещественные колонки, содержащие одно и менее значений. 0 значений мы получаем, когда значения во всех строках Nan.
    numeric_frame_no_const, dropped_const_numeric_columns = remove_constant_features(numeric_frame)
    
    # Удалим категориальные колонки, содержащие ноль значений. Если есть одно значение, то могут быть Nan, которые для
    # категориальных признаков могут быть еще одной категорией (зависит от стратегии обработки).
    categorial_frame_no_const, dropped_const_categorial_columns = remove_constant_features(categorial_frame, 1)
    
    # Восстановим фрейм и вернем вместе с ним список удаленных категориальных колонок.
    return (pd.concat([numeric_frame, categorial_frame], axis=1),
            list(dropped_const_numeric_columns),
            list(dropped_const_categorial_columns))

def process_frame_base_model(train_frame, train_labels, test_frame, test_labels, numeric_features, categorial_features):
    """ Функция строит базовую модель из предыдущей недели """
    
    # Удалим константные колонки из train_frame, и такие-же колонки из test_frame
    train_frame, const_numeric_columns, const_categorial_columns = cleanup_frame_common(
        train_frame,
        numeric_features,
        categorial_features)
    test_frame = test_frame.drop(columns=const_numeric_columns)
    test_frame = test_frame.drop(columns=const_categorial_columns)
    
    numeric_features = numeric_features.drop(const_numeric_columns)
    categorial_features = categorial_features.drop(const_categorial_columns)
    
    # Заполним пропущенные вещественные значения средними
    numeric_train, numeric_test, dropped_numeric = fill_numericna_means(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    numeric_features = numeric_features.drop(dropped_numeric)
    
    # Заполним пропущенные категориальные значения строками "NaV" (Not a value)
    categorial_train = train_frame[categorial_features].fillna("NaV")
    categorial_test = test_frame[categorial_features].fillna("NaV")
    
    # Удалим категориальные колонки с одним единственным значением
    categorial_train, dropped_categorial = remove_constant_features(categorial_train)
    categorial_test = categorial_test.drop(columns=dropped_categorial)
    
    categorial_features = categorial_features.drop(dropped_categorial)
    
    # Список удаленных колонок
    dropped_numeric = np.concatenate([
        list(const_numeric_columns),
        list(dropped_numeric)])
    dropped_categorial = np.concatenate([
        list(const_categorial_columns),
        list(dropped_categorial)])
    
    return (pd.concat([numeric_train, categorial_train], axis=1),
            train_labels,
            pd.concat([numeric_test, categorial_test], axis=1),
            test_labels,
            dropped_numeric,
            dropped_categorial)

def scale_features(train_frame, test_frame):
    train_numeric = train_frame.as_matrix()
    
    scaler = StandardScaler().fit(train_numeric)
    
    train_numeric = coo_matrix(scaler.transform(train_numeric))
    test_numeric = coo_matrix(scaler.transform(test_frame.as_matrix()))
    
    return (train_numeric, test_numeric, scaler)

def one_hot_features(train_frame, test_frame):
    fit_matrix = pd.concat([train_frame, test_frame]).as_matrix()
    
    if fit_matrix.shape[0] == 0 or fit_matrix.shape[1] == 0:
        return (coo_matrix(train_frame.as_matrix()), coo_matrix(test_frame.as_matrix()), None)
    categorial_encoder = CompositeEncoder([MatrixLabelEncoder, OneHotEncoder]).fit(fit_matrix)
    
    train_categorial = categorial_encoder.transform(train_frame.as_matrix())
    test_categorial = categorial_encoder.transform(test_frame.as_matrix())
    
    return (train_categorial, test_categorial, categorial_encoder)

def int_label_features(train_frame, test_frame):
    fit_matrix = pd.concat([train_frame, test_frame]).as_matrix()
    categorial_encoder = MatrixLabelEncoder().fit(fit_matrix)
    
    train_categorial = categorial_encoder.transform(train_frame.as_matrix())
    test_categorial = categorial_encoder.transform(test_frame.as_matrix())
    
    return (train_categorial, test_categorial, categorial_encoder)

def frame_to_matrix_one_hot(train_frame, test_frame, numeric_features, categorial_features):
    """ Функци преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные с помощью OneHotEncoding. """
    
    # Масштабируем вещественные признаки
    train_numeric, test_numeric, scaler = scale_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    # One hot encode для категориальных признаков
    train_categorial, test_categorial, categorial_encoder = one_hot_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            scaler,
            categorial_encoder)

def frame_to_matrix_labeled(train_frame, test_frame, numeric_features, categorial_features):
    """ Функция преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные целыми числами. """
    
    # Масштабируем вещественные признаки
    train_numeric, test_numeric, scaler = scale_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    train_categorial, test_categorial, categorial_encoder = int_label_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            scaler,
            categorial_encoder)

def ridge_baseline_builder(frame, labels, numeric_features, categorial_features):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RidgeClassifier,
        process_frame_base_model,
        frame_to_matrix_one_hot,
        numeric_features,
        categorial_features,
        predict_ridge_proba,
        seed)

def random_forest_baseline_builder(frame, labels, numeric_features, categorial_features):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RandomForestClassifier,
        process_frame_base_model,
        frame_to_matrix_labeled,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

def gradient_boosting_baseline_builder(frame, labels, numeric_features, categorial_features):
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        process_frame_base_model,
        frame_to_matrix_labeled,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

Загрузим train dataset.

In [31]:
churn_data_frame = pd.read_csv("..\..\Data\churn_data_train.csv", ",", dtype= { "Var73": np.float64 })
churn_labels_frame = pd.read_csv("..\..\Data\churn_labels_train.csv", dtype= { "labels": np.int64 })
print(churn_data_frame.shape)
print(churn_labels_frame.shape)

(27999, 230)
(27999, 1)


Выделим числовые и категориальные признаки.

In [4]:
numeric_columns = churn_data_frame.columns[:first_categorial_index]
categorial_columns = churn_data_frame.columns[first_categorial_index:]

Базовые модели

In [5]:
ridge_base = ridge_baseline_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns)
random_forest_base = random_forest_baseline_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns)
gradient_boosting_base = gradient_boosting_baseline_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns)

На 2-й неделе я выбрал в качестве основной метрики F-Score, после 4-й неделе я решил изменить метрику. В качестве основной метрики я буду использовать ROC-AUC. Причина проста. Я максимизирую площадь под ROC кривой, а потом с помощью того-же F-Score могу подобрать оптимальный порог, чтобы максимизировать качество предсказаний.

## Инструкции

1\. Начнем с простого. Давайте оценим как много объектов действительно нужно для построения качественной модели. Для обучения доступна достаточно большая выборка и может так оказаться, что начиная с некоторого момента рост размера обучающей выборки перестает влиять на качество модели. Постройте кривые обучения, обучая модель на выборках разного размера начиная с небольшого количество объектов в обучающей выборке и постепенно наращивая её размер с некоторым шагом. Обратите внимание на `sklearn.model_selection.learning_curve`

In [36]:
def find_best_frame_size(
    pinned_data,
    data,
    best_metric,
    best_size,
    round_size=5):
    step_metric = best_metric
    step_frame, step_labels = data
    step_indices = None
    left_frame = None
    better_metric = best_metric
    better_size = best_size
    
    while step_metric >= best_metric:
        skf = StratifiedKFold(
            n_splits=2,
            shuffle=True,
            random_state=seed)
        step_indices = list(skf.split(step_frame, step_labels))[0][0]
        if(len(step_indices) < 14):
            break
        left_frame = step_frame.drop(step_indices).reset_index(drop=True)
        left_labels = step_labels.drop(step_indices).reset_index(drop=True)
        step_frame = step_frame.loc[step_indices, :].reset_index(drop=True)
        step_labels = step_labels.loc[step_indices, :].reset_index(drop=True)
        
        pinned_frame,pinned_labels = pinned_data
        step_frame = pd.concat([pinned_frame, step_frame], axis=0, ignore_index=True)
        step_labels = pd.concat([pinned_labels, step_labels], axis=0, ignore_index=True).astype(np.int64)
        step_model = gradient_boosting_baseline_builder(
            step_frame,
            step_labels,
            numeric_columns,
            categorial_columns)
        step_metric = np.round(step_model[0], round_size)
        if(step_metric >= better_metric):
            better_metric = step_metric
            better_size = step_frame.shape
        print ("Frame_size %i: %.5f\tInitial quality: %.5f\tLeft_size" % (step_frame.shape[0], step_metric, best_metric, left_frame.shape[0]))
    
    if(len(step_indices) < 14):
        return (better_metric, better_size)
    else:
        return find_best_frame_size(
            (step_frame, step_labels),
            (left_frame, left_labels),
            better_metric,
            better_size)

In [34]:
p_f,p_l = (pd.DataFrame([], columns=churn_data_frame.columns), pd.DataFrame([], columns=churn_labels_frame.columns, dtype=np.int64))
f,l = (churn_data_frame, churn_labels_frame)
s_f = pd.concat([p_f, f], axis=0, ignore_index=True).reset_index(drop=True)
s_l = pd.concat([p_l, l], axis=0, ignore_index=True).reset_index(drop=True).astype(np.int64)

print("Frame")
for col in s_f.columns:
    if(s_f[col].dtype != churn_data_frame[col].dtype):
        print (col, s_f[col].dtype, churn_data_frame[col].dtype)
print("Labels")
for col in s_l.columns:
    print (col, s_l[col].dtype, churn_labels_frame[col].dtype)

Frame
Labels
labels int64 int64


In [ ]:
find_best_frame_size(
    (pd.DataFrame([], columns=churn_data_frame.columns), pd.DataFrame([], columns=churn_labels_frame.columns)),
    (churn_data_frame, churn_labels_frame),
    np.round(gradient_boosting_base[0], 5),
    churn_data_frame.shape)

# Substitute stratified k-fold, to another split logic

Frame_size 13999: 0.71973	Initial quality: 0.73193
Frame_size 20999: 0.71970	Initial quality: 0.73193
Frame_size 24498: 0.73466	Initial quality: 0.73193
Frame_size 33248: 0.75575	Initial quality: 0.73193
Frame_size 37622: 0.77225	Initial quality: 0.73193
Frame_size 39810: 0.77649	Initial quality: 0.73193
Frame_size 40903: 0.78120	Initial quality: 0.73193
Frame_size 41450: 0.78414	Initial quality: 0.73193
Frame_size 41723: 0.78399	Initial quality: 0.73193
Frame_size 41860: 0.78850	Initial quality: 0.73193


2\. Часто несбалансированные по классам выборки приводят к различным проблемам при обучении моделей. Давайте попробуем по-разному обработать выборку, поиграть с распределением объектов по классам и сделать выводы о том, как соотношение классов влияет на качество модели.

2.1\. Задайте веса объектам так, чтобы соотношение классов с учетом весов объектов изменилось. Попробуйте не менее трёх различных вариантов весов. Меняются ли результаты классификации? Как это сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

2.2\. Примените к выборке технологию undersampling: для этого нужно убрать из обучения некоторое количество объектов большего класса таким образом, чтобы соотношение классов изменилось. Попробуйте не менее трёх различных вариантов undersampling (варианты могут отличаться как по количество отфильтрованных объектов, так и по принципу выборка объектов для отсеивания из выборки). Меняются ли результаты классификации? Как это сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

3\. Теперь перейдем к работе с признаками. Ранее вы реализовали несколько стратегий для обработки пропущенных значений. Сравните эти стратегии между собой с помощью оценки качества моделей кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка пропущенных значений сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

4\. Также вы уже реализовали несколько стратегий для обработки категориальных признаков. Сравните эти стратегии между собой с помощью оценки качества моделей по кросс-валидации, построенных на датасетах с использованием различных стратегий. Как обработка категориальных признаков сказывается на качестве модели? Какой вариант выглядит наиболее оптимальным с точки зрения качества?

5\. Все ли признаки оказались полезными для построения моделей? Проведите процедуру отбора признаков, попробуйте разные варианты отбора (обратите внимание на модуль `sklearn.feature_selection`). Например, можно выбрасывать случайные признаки или строить отбор на основе l1-регуляризации - отфильтровать из обучения признаки, которые получат нулевой вес при построении регрессии с l1-регуляризацией (`sklearn.linear_model.Lasso`). И всегда можно придумать что-то своё=) Попробуйте как минимум 2 различные стратегии, сравните результаты. Помог ли отбор признаков улучшить качество модели? Поясните свой ответ.

6\. Подберите оптимальные параметры модели. Обратите внимание, что в зависимости от того, как вы обработали исходные данные, сделали ли балансировку классов, сколько объектов оставили в обучающей выборке и др. оптимальные значения параметров могут меняться. Возьмите наилучшее из ваших решений на текущий момент и проведите процедуру подбора параметров модели (обратите внимание на `sklearn.model_selection.GridSearchCV`) Как подбор параметров повлиял на качество модели?

7\. Предложите методику оценки того, какие признаки внесли наибольший вклад в модель (например, это могут быть веса в случае регрессии, а также большое количество моделей реализуют метод `feature_importances_` - оценка важности признаков). На основе предложенной методики проанализируйте, какие признаки внесли больший вклад в модель, а какие меньший?

8\. Напоследок давайте посмотрим на объекты. На каких объектах достигается наибольшая ошибка классификации? Есть ли межу этими объектами что-то общее? Видны ли какие-либо закономерности? Предположите, почему наибольшая ошибка достигается именно на этих объектах. В данном случае "наибольшую" ошибку можно понимать как отнесение объекта с чужому классу с большой долей уверенности (с высокой вероятностью).

9\. По итогам проведенных экспериментов постройте финальную решение - модель с наилучшим качеством. Укажите, какие преобразования данных, параметры и пр. вы выбрали для построения финальной модели.

10\. Подумайте, можно ли еще улучшить модель? Что для этого можно сделать? 